In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests
from csv import writer
import re
import seaborn as sns
import matplotlib.pyplot as plt

pd.set_option('display.max_columns',100)

import warnings
warnings.filterwarnings("ignore")

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor

from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score


In [3]:
def algo_test(x,y):        
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        GBR=GradientBoostingRegressor()
        XGBC=XGBRegressor()
        kn=KNeighborsRegressor()
        
        x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.1,random_state=13)
        
        algos=[R,Lass,E,GBR,XGBC,kn]
        algo_names=['Ridge','Lasso','ElasticNet','Gradient Boosting','XGradientBoosting','KNeighbors']
        
        r2Score = []
        rmse = []
        mae = []
        
        result=pd.DataFrame(columns=['R2_score','RMSE','MAE'],index=algo_names)
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r2Score.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
            
        result['R2_score']=r2Score 
        result.RMSE=rmse
        result.MAE=mae

        return result.sort_values('R2_score',ascending=False)

In [56]:
def house_scraping(sehir, max_page):
    df=pd.DataFrame(columns=['Kiralik', 'Tanim','Oda','Alan', 'Yas', 'Kat', 'Fiyat'])
    kiralik, tanim, oda, alan, yas, kat, konum, fiyat = [], [], [], [], [], [], [], []
    
    for page in range(max_page):
        response = requests.get(f"https://www.hepsiemlak.com/{sehir}-kiralik?page={page+1}")
        soup = BeautifulSoup(response.content,'html.parser')
        lists = soup.find_all('li',class_="listing-item")
    
        for i in lists:
            kiralik.append(i.find('div',class_ = 'left').text.replace('\n',''))
            tanim.append(i.find('header',class_ = 'list-view-header').text.replace('\n',''))
            oda.append(i.find('span',class_ = 'celly houseRoomCount').text.replace('\n',''))
            alan.append(i.find('span',class_ = 'celly squareMeter list-view-size').text.replace('\n',''))
            yas.append(i.find('span',class_ = 'celly buildingAge').text.replace('\n',''))
            kat.append(i.find('span',class_ = 'celly floortype').text.replace('\n',''))
            konum.append(i.find('div',class_ = 'list-view-location').text.replace('\n',''))
            fiyat.append(i.find('span',class_ = 'list-view-price').text.replace('\n',''))
            
    df['Kiralik']=kiralik        
    df['Tanim']=tanim
    df['Oda']=oda
    df['Alan']=alan
    df['Yas']=yas
    df['Kat']=kat
    df['Konum']=konum
    df['Fiyat']=fiyat
    return df

In [5]:
city='cankaya'

In [58]:
df = house_scraping(city, 107) 

In [59]:
# bosluklar for dongusunun icinde silinirse hesaplama sursi cok uzar bu yuzden burada silecegiz
f = lambda x: " ".join(x.split())
for i in df.columns:
    df[i]=df[i].apply(f)

In [60]:
df

,Kiralik,Tanim,Oda,Alan,Yas,Kat,Fiyat,Konum
0,Kiralık Daire,,3 + 1,135 m2,28 Yaşında,1. Kat,13.000 TL,"Çankaya, Bahçelievler Mahallesi"
1,Kiralık Daire,öğrencilere sokulluda 5+2 yapılı möbleli kiral...,4 + 2,300 m2,25 Yaşında,Teras Katı,12.500 TL,"Çankaya, Harbiye Mahallesi"
2,Kiralık Daire,"panora yakını, muhteşem manzaralı,4+2 dubleks,...",4 + 2,220 m2,13 Yaşında,3. Kat,8.000 TL,"Çankaya, Mürsel Uluç Mahallesi"
3,Kiralık Daire,"panora yakını, muhteşem 4+2, mobilyalı.75 m2…t...",4 + 2,220 m2,13 Yaşında,3. Kat,10.000 TL,"Çankaya, Mürsel Uluç Mahallesi"
4,Kiralık Daire,vesta...bölgede marka olmuş ultralüx sitede-gü...,4 + 2,350 m2,4 Yaşında,15. Kat,15.500 TL,"Çankaya, Bademlidere Mahallesi"
...,...,...,...,...,...,...,...,...
2563,Kiralık Daire,furnıshed flat ın cankaya wıth terrace,3 + 1,120 m2,40 Yaşında,Teras Katı,18.000 TL,"Çankaya, Çankaya Mahallesi"
2564,Kiralık Bina,mavi emlak'tan gop hülya sokak'ta komple kiral...,7 + 1,750 m2,25 Yaşında,,88.000 TL,"Çankaya, Gaziosmanpaşa Mahallesi"
2565,Kiralık Daire,furnıshed apartment ın kavaklıdere,3 + 1,150 m2,40 Yaşında,Ara Kat,27.000 TL,"Çankaya, Barbaros Mahallesi"
2566,Kiralık Daire,unfurnıshed 5+1+terrace dublex apartment ın yı...,5 + 1,370 m2,17 Yaşında,3. Kat,18.000 TL,"Çankaya, Yıldızevler Mahallesi"


In [61]:
df.to_csv(city+'.csv', encoding='utf-8-sig', index=False)

In [6]:
df = pd.read_csv(city+'.csv', encoding='utf-8-sig')

## EDA

In [7]:
df[df.Kat.isnull()].Kiralik.value_counts()

Kiralık Villa          131
Kiralık Bina            10
Kiralık Müstakil Ev      2
KiralÄ±k Villa           1
Name: Kiralik, dtype: int64

In [8]:
df.head(10)

,Kiralik,Tanim,Oda,Alan,Yas,Kat,Fiyat,Konum
0,Kiralık Daire,NaN,3 + 1,135 m2,28 Yaşında,1. Kat,13.000 TL,"Çankaya, Bahçelievler Mahallesi"
1,Kiralık Daire,özel bahçeli özel garajlı kedi köpek beslemeye...,3 + 1,135 m2,7 Yaşında,Bahçe Katı,5.000 TL,"Çankaya, Bağcılar Mahallesi"
2,Kiralık Daire,öğrencilere sokulluda 5+2 yapılı möbleli kiral...,4 + 2,300 m2,25 Yaşında,Teras Katı,12.500 TL,"Çankaya, Harbiye Mahallesi"
3,Kiralık Daire,"panora yakını, muhteşem manzaralı,4+2 dubleks,...",4 + 2,220 m2,13 Yaşında,3. Kat,8.000 TL,"Çankaya, Mürsel Uluç Mahallesi"
4,Kiralık Daire,"panora yakını, muhteşem 4+2, mobilyalı.75 m2…t...",4 + 2,220 m2,13 Yaşında,3. Kat,10.000 TL,"Çankaya, Mürsel Uluç Mahallesi"
5,Kiralık Daire,vesta...bölgede marka olmuş ultralüx sitede-gü...,4 + 2,350 m2,4 Yaşında,15. Kat,15.500 TL,"Çankaya, Bademlidere Mahallesi"
6,Kiralık Daire,ets'den cevizlidere cad yakını bakımlı 4+1 dub...,4 + 1,170 m2,5 Yaşında,3. Kat,6.600 TL,"Çankaya, Huzur Mahallesi"
7,Kiralık Daire,çayyolu merkezde arcadium metro yakını 3+1,3 + 1,130 m2,25 Yaşında,4. Kat,5.800 TL,"Çankaya, Koru Mahallesi"
8,Kiralık Daire,portakal çiçeği vadisi sitesinde 4+1 yapılı mu...,4 + 1,300 m2,15 Yaşında,14. Kat,25.000 TL,"Çankaya, Aziziye Mahallesi"
9,Kiralık Daire,vıp desıgned fully furnıshed loft resıdence ın...,6 + 1,375 m2,5 Yaşında,Ara Kat,45.000 TL,"Çankaya, Kızılırmak Mahallesi"


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2568 entries, 0 to 2567
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Kiralik  2568 non-null   object
 1   Tanim    2567 non-null   object
 2   Oda      2568 non-null   object
 3   Alan     2568 non-null   object
 4   Yas      2568 non-null   object
 5   Kat      2424 non-null   object
 6   Fiyat    2568 non-null   object
 7   Konum    2568 non-null   object
dtypes: object(8)
memory usage: 160.6+ KB


In [10]:
df.isnull().mean() # aslinda bos var ancak '' seklinde

Kiralik    0.000000
Tanim      0.000389
Oda        0.000000
Alan       0.000000
Yas        0.000000
Kat        0.056075
Fiyat      0.000000
Konum      0.000000
dtype: float64

In [11]:
pd.DataFrame(df=='').mean() # Kat verisinin %33u bos

Kiralik    0.0
Tanim      0.0
Oda        0.0
Alan       0.0
Yas        0.0
Kat        0.0
Fiyat      0.0
Konum      0.0
dtype: float64

### Kiralik

In [12]:
df.Kiralik.value_counts()

Kiralık Daire          2287
Kiralık Villa           146
Kiralık Residence        95
KiralÄ±k Daire           23
Kiralık Bina             10
Kiralık Loft Daire        3
Kiralık Müstakil Ev       2
KiralÄ±k Villa            1
Kiralık Prefabrik         1
Name: Kiralik, dtype: int64

### Oda

In [13]:
df.Oda.value_counts()

3 + 1      946
4 + 1      553
2 + 1      452
1 + 1      243
5 + 1      120
5 + 2       51
6 + 1       49
4 + 2       34
6 + 2       24
7 + 1       24
Stüdyo      18
7 + 2       13
3 + 2       13
8 + 1        3
2 + 2        3
5 + 3        2
7 + 3        2
4 + 3        1
1 + 3        1
8 + 3        1
6 + 25       1
11 + 2       1
9 + 1        1
11 + 11      1
15 + 3       1
4 + 4        1
9 + 3        1
6 + 6        1
10 + 3       1
17 + 1       1
3 + 0        1
20 + 1       1
30 + 5       1
6 + 3        1
9 + 2        1
Name: Oda, dtype: int64

In [14]:
# delete outliers
a=['6 + 25', '11 + 2', '11 + 11', '15 + 3', '10 + 3', '17 + 1', '20 + 1', '30 + 5']
for i in a:
    df.drop(df[df.Oda==i].index, inplace=True) 

In [15]:
df[df.Oda=='Stüdyo'].Tanim.replace({'[^\b\d+\b]':''}, regex=True) # studyolar genelde 1 + 0

249                 
476                 
516                 
529              1+0
593                 
793           3121+0
1376    1+0251521552
1398          3121+0
1410             1+0
1815                
1840               1
2042             1+0
2223                
2293             1+0
2310             1+0
2456                
2474             1+0
2479             1+0
Name: Tanim, dtype: object

In [16]:
df.Oda.replace({'Stüdyo':'1 + 0'}, regex=True, inplace=True)

In [17]:
# Oda syisini sayiya ceviricez, kat ondalik, salon birlik basamaginda olacak
df.Oda.replace({'[^\b\d\b]':''}, regex=True, inplace=True)
df['Oda'] = df.Oda.astype('float64')

### Alan

In [18]:
df['Alan'] = df['Alan'].replace({' m2':''},regex=True).astype('float64')

In [19]:
df.Alan

0       135.0
1       135.0
2       300.0
3       220.0
4       220.0
        ...  
2563    200.0
2564    120.0
2565    750.0
2566    150.0
2567    370.0
Name: Alan, Length: 2560, dtype: float64

### Yas

In [20]:
df.Yas.value_counts()
# cok yasli binalar gercek, tas evler vb.

Sıfır Bina      213
25 Yaşında      208
30 Yaşında      200
20 Yaşında      190
15 Yaşında      149
               ... 
17 YaÅŸÄ±nda      1
40 YaÅŸÄ±nda      1
25 YaÅŸÄ±nda      1
43 YaÅŸÄ±nda      1
53 Yaşında        1
Name: Yas, Length: 67, dtype: int64

In [21]:
df['Yas'].replace({'[^\b\d\b]':''}, regex=True, inplace=True) # Sadece sayilar alinir

In [22]:
df.loc[df.Yas=='', 'Yas']= '0' # Sifir yasinda yazanlar yok oldugundan simdi atadik

In [23]:
df['Yas'] = df['Yas'].astype('float64')

In [24]:
df.Yas.value_counts()

0.0      214
25.0     209
30.0     203
20.0     192
15.0     153
10.0     148
35.0     140
5.0      126
40.0     107
4.0      103
1.0       88
2.0       64
3.0       62
12.0      52
11.0      47
45.0      45
16.0      45
8.0       44
7.0       42
13.0      39
18.0      37
50.0      34
22.0      32
26.0      30
6.0       27
28.0      27
27.0      25
21.0      23
17.0      23
9.0       22
31.0      21
36.0      19
14.0      17
23.0      16
24.0      16
32.0      13
19.0       9
29.0       8
33.0       7
38.0       6
55.0       6
42.0       4
39.0       3
37.0       2
47.0       2
43.0       1
34.0       1
256.0      1
48.0       1
59.0       1
60.0       1
41.0       1
53.0       1
Name: Yas, dtype: int64

### Kat

In [25]:
df.Kat.fillna('0. Kat', inplace=True) # Nan larin cogu villa

In [26]:
df.Kat.value_counts(dropna=False)

2. Kat          322
3. Kat          289
1. Kat          288
Ara Kat         218
4. Kat          155
Yüksek Giriş    154
0. Kat          137
21 ve üzeri     108
Bahçe Katı       93
En Üst Kat       72
5. Kat           71
Giriş Katı       70
Kot 1            67
7. Kat           58
8. Kat           55
6. Kat           55
9. Kat           50
12. Kat          31
10. Kat          31
11. Kat          28
Kot 2            23
13. Kat          22
14. Kat          20
Teras Katı       18
17. Kat          17
20. Kat          16
Çatı Katı        13
16. Kat          12
Kot 3            12
Zemin            11
19. Kat          10
15. Kat          10
18. Kat           9
Villa Katı        8
Bodrum            3
Yarı Bodrum       2
21 ve Ã¼zeri      1
Asma Kat          1
Name: Kat, dtype: int64

In [27]:
#df['Kat'].replace({'Zemin': '0. Kat', 'Kot 2':'2. Kat', 'Kot 1':'1. Kat', 'Kot 3':'3. Kat', 
#                   'Bodrum':'0. Kat', 'Yarı Bodrum':'0. Kat', 'Bahçe Katı':'0. Kat',
#                   'Villa Katı': '3. Kat', 'Ara Kat':'7. Kat', 'Yüksek Giriş':'0. Kat',
#                   'Giriş Katı': '0. Kat', 'Çatı Katı': '15. Kat', 'Teras Katı':'15. Kat',
#                   '21 ve üzeri' :'21. Kat', 'En Üst Kat':'15. Kat'}, inplace=True)

In [28]:
#df['Kat_int'] = df['Kat'].replace({'[^\b\d\b]':''}, regex=True) # kacinci kat oldugunu veren kod

In [29]:
#df['Kat_int'] = df['Kat_int'].astype(int)

### Fiyat

In [30]:
df['Fiyat'] = df['Fiyat'].replace({'[^\b\d+\b]':''}, regex=True).astype('float64')

In [31]:
df.Fiyat.value_counts()

6000.0       105
5000.0        95
7500.0        95
5500.0        90
10000.0       79
            ... 
8850.0         1
1500.0         1
2350000.0      1
8400.0         1
88000.0        1
Name: Fiyat, Length: 271, dtype: int64

### Konum

In [32]:
df.Konum.value_counts()

Çankaya, Oran Mahallesi             138
Çankaya, Alacaatlı Mahallesi        129
Çankaya, Büyükesat Mahallesi        118
Çankaya, İlkbahar Mahallesi          91
Çankaya, Aziziye Mahallesi           83
                                   ... 
Ã‡ankaya, KÃ¼Ã§Ã¼kesat Mahallesi      1
Ã‡ankaya, Ä°lkbahar Mahallesi         1
Ã‡ankaya, Ã–veÃ§ler Mahallesi         1
Ã‡ankaya, Cebeci Mahallesi            1
Ã‡ankaya, Mutlukent Mahallesi         1
Name: Konum, Length: 131, dtype: int64

In [33]:
df['Konum'] = df.Konum.str.split(',').str[1]

### Outliers

In [34]:
outliars0 = df.quantile(.9)
outliars1 = df.quantile(.1)

cols = ['Fiyat', 'Oda', 'Alan', 'Yas']
for i in cols:
  df = df[(df[i]<outliars0[i]) & (df[i]>outliars1[i])]

## Regression

In [35]:
df.head()

,Kiralik,Tanim,Oda,Alan,Yas,Kat,Fiyat,Konum
0,Kiralık Daire,NaN,31.0,135.0,28.0,1. Kat,13000.0,Bahçelievler Mahallesi
3,Kiralık Daire,"panora yakını, muhteşem manzaralı,4+2 dubleks,...",42.0,220.0,13.0,3. Kat,8000.0,Mürsel Uluç Mahallesi
4,Kiralık Daire,"panora yakını, muhteşem 4+2, mobilyalı.75 m2…t...",42.0,220.0,13.0,3. Kat,10000.0,Mürsel Uluç Mahallesi
6,Kiralık Daire,ets'den cevizlidere cad yakını bakımlı 4+1 dub...,41.0,170.0,5.0,3. Kat,6600.0,Huzur Mahallesi
7,Kiralık Daire,çayyolu merkezde arcadium metro yakını 3+1,31.0,130.0,25.0,4. Kat,5800.0,Koru Mahallesi


In [36]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1134 entries, 0 to 2563
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   Kiralik  1134 non-null   object 
 1   Tanim    1133 non-null   object 
 2   Oda      1134 non-null   float64
 3   Alan     1134 non-null   float64
 4   Yas      1134 non-null   float64
 5   Kat      1134 non-null   object 
 6   Fiyat    1134 non-null   float64
 7   Konum    1134 non-null   object 
dtypes: float64(4), object(4)
memory usage: 79.7+ KB


In [37]:
df.nunique()

Kiralik       5
Tanim      1098
Oda           7
Alan         80
Yas          33
Kat          36
Fiyat       127
Konum       101
dtype: int64

In [38]:
df_dummies = pd.get_dummies(df.drop(columns=['Tanim']), drop_first=True)
cols = df_dummies.columns.to_list()

In [39]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
df_dummies=scaler.fit_transform(df_dummies)
df_dummies=pd.DataFrame(df_dummies, columns=cols)

In [40]:
x=df_dummies.drop(columns=['Fiyat'])
y=df_dummies['Fiyat']

In [41]:
algo_test(x,y)

,R2_score,RMSE,MAE
Ridge,0.628959,0.162641,0.123957
Gradient Boosting,0.626289,0.163225,0.126483
XGradientBoosting,0.577125,0.173630,0.127707
KNeighbors,0.547376,0.179634,0.143690
Lasso,-0.021108,0.269808,0.231168
ElasticNet,-0.021108,0.269808,0.231168


In [42]:
df.corr()

,Oda,Alan,Yas,Fiyat
Oda,1.000000,0.745737,-0.014328,0.228305
Alan,0.745737,1.000000,-0.177249,0.471883
Yas,-0.014328,-0.177249,1.000000,-0.327004
Fiyat,0.228305,0.471883,-0.327004,1.000000


## Deep Learning

In [43]:
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=.2, random_state=7)

In [44]:
from tensorflow.keras.layers import Dense, Flatten, Dropout, LSTM, Embedding, Bidirectional, BatchNormalization
model=Sequential()
model.add(Dense(81, activation='relu'))
model.add(Dense(40, activation='relu'))
model.add(Dense(27, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(19, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(3, activation='relu'))
model.add(Dense(1))
model.compile(optimizer='Adam', loss='mse')

In [45]:
model.fit(x=x_train,y=y_train,validation_data=(x_test,y_test),verbose=1,epochs=30 )

Epoch 1/30
29/29 [==============================] - 1s 13ms/step - loss: 0.0946 - val_loss: 0.0595
Epoch 2/30
29/29 [==============================] - 0s 6ms/step - loss: 0.0593 - val_loss: 0.0459
Epoch 3/30
29/29 [==============================] - 0s 10ms/step - loss: 0.0454 - val_loss: 0.0353
Epoch 4/30
29/29 [==============================] - 0s 5ms/step - loss: 0.0353 - val_loss: 0.0297
Epoch 5/30
29/29 [==============================] - 0s 5ms/step - loss: 0.0306 - val_loss: 0.0281
Epoch 6/30
29/29 [==============================] - 0s 5ms/step - loss: 0.0253 - val_loss: 0.0274
Epoch 7/30
29/29 [==============================] - 0s 5ms/step - loss: 0.0220 - val_loss: 0.0273
Epoch 8/30
29/29 [==============================] - 0s 5ms/step - loss: 0.0211 - val_loss: 0.0274
Epoch 9/30
29/29 [==============================] - 0s 5ms/step - loss: 0.0205 - val_loss: 0.0282
Epoch 10/30
29/29 [==============================] - 0s 6ms/step - loss: 0.0181 - val_loss: 0.0280
Epoch 11/30
29/29

In [46]:
pred=model.predict(x_test)

8/8 [==============================] - 0s 2ms/step


In [47]:
(mean_squared_error(y_test, pred))**.5

0.17928778287780203

In [48]:
r2_score(y_test, pred)

0.476604740568921